In [ ]:
import preamble

# Constants

In [ ]:
import numpy as np

# Experiment
USER_ID = 0

# Trajectory Planner
REFERENCE_TRAJECTORY_CSV_PATH = '../data/reference_path.csv'
INIT_POSE = np.array([-0.54, -0.27, 0.0, -1.21, -1.21, 1.25]) # Also used for Robot

# Physical PD
PHYSICAL_PD_SCALER = 2
K_p = 19 * PHYSICAL_PD_SCALER
B_p = 11 * PHYSICAL_PD_SCALER

# Physical Admittance
M = 2
B = 17.7

# Real time data collection
TABLE_COLUMN_NAMES = [
    't',
    'x',
    'x_dot',
    'F_h',
    'ref_going_forward',
    'x_ref',
    'x_dot_ref',
    'e',
    'e_dot',
    'P_hat',
    'V_hat',
    'c_p',
    'c_v',
    'verbal_P_tracking_term',
    'verbal_D_tracking_term',
    'verbal_PD_tracking_term',
    'A_v',
    'U_v',
    'physical_P_tracking_term',
    'physical_D_tracking_term',
    'physical_PD_tracking_term',
    'A_p',
    'U_p',
    'successful_utterance',
    'is_speaking',
    'phrase',
    'U',
    'human_disturbance',
    'system_dynamics_input',
]
DATA_COLLECTION_SAMPLING_RATE = 200

# Real time data plotting
REAL_TIME_DATA_PLOTTING_ENABLED = True
PLOTTING_SAMPLING_RATE = 30

# Robot
ROBOT_IP = '169.254.9.43'
FORCE_EWMA_TAU = 0.5
TRANSLATIONAL_FORCE_DEADBAND = 2

# Experiment

This baseline is made up of all componentes: Physical PD, Physical Admittance, Verbal PD, Verbal Admittance, F2L, Compliance Estimator, and $J$ Optimization Method.

In [ ]:
from timer import Timer
from trajectory_planner import TrajectoryPlanner
from model import SVMKNNModel
import pickle
from vocalizer import Vocalizer
from virtual_dynamics import SimpleVirtualDynamics
from tabular_data_store import TabularDataStore
from plot_client import PlotClient
from robot import Robot
from console_command_thread import ConsoleCommandThread
import scipy.stats
from phrase_mapping import PHRASE_MAPPING, MEAN_WORDS_PER_PHRASE

#############
### SETUP ###
#############

### Experiment ###
experiment_timer = Timer()

### Trajectory Planner ###
traj = TrajectoryPlanner(REFERENCE_TRAJECTORY_CSV_PATH, INIT_POSE[:3])

### System dynamics ###
vd = SimpleVirtualDynamics(M=M, B=B, K=0)

### Real time data collection ###
table = TabularDataStore(column_names=TABLE_COLUMN_NAMES)
table_timer = Timer()

### Real time data plotting ###
if REAL_TIME_DATA_PLOTTING_ENABLED:
    plot = PlotClient()

    plot.create_plot("force_plot", title="Force vs Time", xlabel="Time (s)", ylabel="Force")
    plot.create_line("force_plot", "Fx", color="r", label="Fx")
    plot.create_line("force_plot", "Fy", color="g", label="Fy")
    plot.create_line("force_plot", "Fz", color="b", label="Fz")

    plot.create_plot("velocity_plot", title="Velocity vs Time", xlabel="Time (s)", ylabel="Velocity")
    plot.create_line("velocity_plot", "vx", color="r", label="vx")
    plot.create_line("velocity_plot", "vy", color="g", label="vy")
    plot.create_line("velocity_plot", "vz", color="b", label="vz")

    plot.create_plot("error_plot", title="Error vs Time", xlabel="Time (s)", ylabel="Error")
    plot.create_line("error_plot", "ex", color="r", label="ex")
    plot.create_line("error_plot", "ey", color="g", label="ey")
    plot.create_line("error_plot", "ez", color="b", label="ez")
    
    plot.create_plot("control_plot", title="Control Inputs vs Time", xlabel="Time (s)", ylabel="Control Norm")
    plot.create_line("control_plot", "U_p", color="cyan", label="‖U_p‖")

    plot_timer = Timer()

e_integral = 0

### Robot ###
with Robot(
    ROBOT_IP,
    default_axes=Robot.TRANSLATION,
    init_pose=INIT_POSE,
    force_ewma_tau=FORCE_EWMA_TAU,
    translational_force_deadband=TRANSLATIONAL_FORCE_DEADBAND,
) as r, ConsoleCommandThread() as c:

    ##################
    ### EXPERIMENT ###
    ##################

    experiment_timer.reset()

    while c.is_alive():
        period_start = r.control.initPeriod()

        ### Current state ###
        t = experiment_timer.t()
        dt = experiment_timer.dt()
        x = r.get_pose()
        x_dot = r.get_velocity()
        F_h = r.get_force()

        ### Trajectory Planner ###
        traj.update_reference_trajectory(x)
        x_ref, x_dot_ref = traj.get_closest_target(x)

        ### Tracking errors ###
        e = x_ref - x
        e_dot = x_dot_ref - x_dot

        ### Physical PD ###
        physical_P_tracking_term = K_p * e
        physical_D_tracking_term = B_p * e_dot
        physical_PD_tracking_term = physical_P_tracking_term + physical_D_tracking_term

        ### Physical Admittance ###
        A_p = 0.0
        U_p = physical_PD_tracking_term - A_p*F_h

        ### System dynamics ###
        U = U_p + 0
        human_disturbance = F_h - 0
        system_dynamics_input = U + human_disturbance
        vd.apply_force(system_dynamics_input, dt)
        r.set_velocity(vd.get_velocity(), acceleration=10)

        ### Real time data collection ###
        if table_timer.t() >= DATA_COLLECTION_SAMPLING_RATE**(-1):
            table.append_row((
                t,
                x,
                x_dot,
                F_h,
                np.nan,
                x_ref,
                x_dot_ref,
                e,
                e_dot,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                physical_P_tracking_term,
                physical_D_tracking_term,
                physical_PD_tracking_term,
                A_p,
                U_p,
                np.nan,
                np.nan,
                np.nan,
                U,
                human_disturbance,
                system_dynamics_input,
            ))
            
            table_timer.reset()
            
        ### Real time data plotting ###
        if REAL_TIME_DATA_PLOTTING_ENABLED:
            if plot_timer.t() >= PLOTTING_SAMPLING_RATE**(-1):
                plot.update_line("force_plot", "Fx", (t, F_h[0]))
                plot.update_line("force_plot", "Fy", (t, F_h[1]))
                plot.update_line("force_plot", "Fz", (t, F_h[2]))

                plot.update_line("velocity_plot", "vx", (t, x_dot[0]))
                plot.update_line("velocity_plot", "vy", (t, x_dot[1]))
                plot.update_line("velocity_plot", "vz", (t, x_dot[2]))

                plot.update_line("error_plot", "ex", (t, e[0]))
                plot.update_line("error_plot", "ey", (t, e[1]))
                plot.update_line("error_plot", "ez", (t, e[2]))

                plot.update_line("control_plot", "U_p", (t, np.linalg.norm(U_p)))

                plot_timer.reset()

            r.control.waitPeriod(period_start)

table.to_pandas().to_pickle(f'../data/experiments/redoing_state_estimator/{USER_ID}.pkl')

In [ ]:
import numpy as np

df = table.to_pandas()
import matplotlib.pyplot as plt

U_p_magnitude = df['U_p'].apply(lambda x: np.linalg.norm(x))
plt.figure(figsize=(10, 5))
plt.plot(df['t'], U_p_magnitude)
plt.xlabel('Time (s)')
plt.ylabel('‖U_p‖')
plt.title('Magnitude of U_p over Time')
plt.grid(True)
plt.show()